# Bedrock Model Routing - LLM routing

## Intro and Goal
This Jupyter Notebook is designed to test an LLM (Large Language Model) routing system. The goal is to take a prompt, embed it using a vector embedding in Bedrock, and then measure the distance with two specific vectors that represent the domain for two specific LLMs. Based on the distance, the prompt will be routed to the appropriate LLM.

The notebook is structured as follows:
1. Create the samples for the 2 domains that we'll route to (e.g., code generation and summarization).
2. Generate the embeddings for the 2 domain prompts.
3. Create a 3rd prompt, generate its embedding, and measure the distance to select which domain it relates to.
4. Construct the router that will take the prompt and automatically generate the answer from the LLM the prompt is routed to based on the distance.

In [41]:
# Import necessary libraries
import boto3
import json

In [55]:
# Step 1: Define your LLM router
router_prompt = "Give this question a difficulty rating, from 1 to 3, simply provide the number without anything else in your answer:\n"
router_model = "anthropic.claude-3-haiku-20240307-v1:0"

In [62]:
# Step 2: Evaluate the prompt
prompt = "Implement a function that sorts a list of integers in descending order using the insertion sort algorithm."
brt = boto3.client(service_name='bedrock-runtime', region_name="us-west-2")
from botocore.exceptions import ClientError
# Format the request payload using the model's native structure.
def eval(prompt):
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": router_prompt + prompt}],
            }
        ],
    }
    
    # Convert the native request to JSON.
    request = json.dumps(native_request)
    
    try:
        # Invoke the model with the request.
        response = client.invoke_model(modelId=router_model, body=request)
    
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)
    
    # Decode the response body.
    model_response = json.loads(response["body"].read())
    
    # Extract and print the response text.
    response_text = model_response["content"][0]["text"]
    return response_text

In [63]:
eval(prompt)

'2'

In [68]:
# Step 3: Define your model selection
model_1 = "anthropic.claude-3-haiku-20240307-v1:0"
native_request_1 = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "temperature": 0.5,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
}
model_2 = "anthropic.claude-3-5-sonnet-20240620-v1:0"
native_request_2 = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "temperature": 0.5,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}],
        }
    ],
}
model_3 = "meta.llama3-1-70b-instruct-v1:0"
native_request_3 = {
    "prompt": prompt,
    "max_gen_len": 512,
    "temperature": 0.5,
}

In [71]:
# Step 4: Construct the router
def route_prompt(prompt):
    if eval(prompt)==1:
        request = json.dumps(native_request_1)
        try:
            # Invoke the model with the request.
            response = client.invoke_model(modelId=model_1, body=request)
        
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{model_1}'. Reason: {e}")
            exit(1)
        
        # Decode the response body.
        model_response = json.loads(response["body"].read())
        
        # Extract and print the response text.
        response_text = model_response["content"][0]["text"]
        return response_text
    if eval(prompt)==2:
        request = json.dumps(native_request_2)
        try:
            # Invoke the model with the request.
            response = client.invoke_model(modelId=model_2, body=request)
        
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{model_2}'. Reason: {e}")
            exit(1)
        
        # Decode the response body.
        model_response = json.loads(response["body"].read())
        
        # Extract and print the response text.
        response_text = model_response["content"][0]["text"]
        return response_text
    else:
        request = json.dumps(native_request_3)
        try:
            # Invoke the model with the request.
            response = client.invoke_model(modelId=model_3, body=request)
        
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{model_3}'. Reason: {e}")
            exit(1)
        
        # Decode the response body.
        model_response = json.loads(response["body"].read())
        
        # Extract and print the response text.
        response_text = model_response["generation"]
        return response_text

In [72]:
route_prompt("Implement a function that sorts a list of integers in descending order using the insertion sort algorithm.")

'**\n\n### Solution\n```python\ndef insertion_sort_descending(lst):\n    """\n    Sorts a list of integers in descending order using the insertion sort algorithm.\n\n    Args:\n        lst (list): A list of integers.\n\n    Returns:\n        list: The sorted list in descending order.\n    """\n    # Iterate through the list starting from the second element (index 1)\n    for i in range(1, len(lst)):\n        # Store the current element to be inserted\n        current_element = lst[i]\n        \n        # Initialize the index of the previous element\n        j = i - 1\n        \n        # Shift elements to the right until a smaller element is found\n        while j >= 0 and lst[j] < current_element:\n            # Shift the element to the right\n            lst[j + 1] = lst[j]\n            # Move to the previous element\n            j -= 1\n        \n        # Insert the current element at the correct position\n        lst[j + 1] = current_element\n    \n    # Return the sorted list\n  

# other
Dataset of questions on specific difficulty and topics
zero shot vs few shot vs fine-tuned
create a synthetic dataset (text to sql)